### Get the MNIST Data

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Helper Functions

In [3]:
def init_weights(shape): #helps intialize weights for a layer
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape): #helps intialize biases for a layer
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [4]:
def conv2d(x,W):
    # x --> [Batch,Height,Width,Channels] #Input Tensor
    # W --> [filter Height,filter Width,Channels_IN,Channels_OUT]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [5]:
def max_pool(x):
    # x --> [batch,h,w,c]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [6]:
def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b) #Using a Rectified Linear Unit(ReLu) activation function.

In [7]:
def fully_connected_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W)+b

In [8]:
#PlaceHolders
x = tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [9]:
#Layers
x_image = tf.reshape(x,[-1,28,28,1])

convo1 = convolutional_layer(x_image,shape=[4,4,1,32]) #4x4 filter, 1 is from the original image, 32 outputs.
convo1_pooling = max_pool(convo1)

convo2 = convolutional_layer(convo1_pooling,shape=[4,4,32,64])
convo2_pooling = max_pool(convo2)

convo2_flat = tf.reshape(convo2_pooling,[-1,7*7*64]) #7x7 cuz we did 2 pooling layers, so (28/2)/2=7 
full_layer1 = tf.nn.relu(fully_connected_layer(convo2_flat,1024))

hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer1,keep_prob=hold_prob)

y_pred = fully_connected_layer(full_one_dropout,10)

In [10]:
#Loss Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [11]:
#Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001) #Using Adam Optimizer
train = optimizer.minimize(cross_entropy)

In [12]:
init = tf.global_variables_initializer()

### Session

In [13]:
steps = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x , batch_y = mnist.train.next_batch(50)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')

Currently on step 0
Accuracy is:
0.0987


Currently on step 100
Accuracy is:
0.9428


Currently on step 200
Accuracy is:
0.9574


Currently on step 300
Accuracy is:
0.9707


Currently on step 400
Accuracy is:
0.976


Currently on step 500
Accuracy is:
0.9783


Currently on step 600
Accuracy is:
0.9786


Currently on step 700
Accuracy is:
0.9815


Currently on step 800
Accuracy is:
0.984


Currently on step 900
Accuracy is:
0.9825


Currently on step 1000
Accuracy is:
0.9844


Currently on step 1100
Accuracy is:
0.9807


Currently on step 1200
Accuracy is:
0.9826


Currently on step 1300
Accuracy is:
0.9874


Currently on step 1400
Accuracy is:
0.9827


Currently on step 1500
Accuracy is:
0.9869


Currently on step 1600
Accuracy is:
0.9861


Currently on step 1700
Accuracy is:
0.9876


Currently on step 1800
Accuracy is:
0.9877


Currently on step 1900
Accuracy is:
0.9886


Currently on step 2000
Accuracy is:
0.9877


Currently on step 2100
Accuracy is:
0.988


Currently on step 2200
Ac